# Использование сегмента базы знаний, сформированного в первой части работы

# Цель работы
Разработка проекта сегмента базы знаний интеллектуальной системы, содержащей набор моделей, сформированных с помощью различных методов машинного обучения (модели линейной множественной регрессии, деревья решений, нейронные сети). 

Знакомство с технологией пополнения базы знаний и её использования.

In [1]:
import pandas as pd
import numpy as np

###import pybrain as pb
from pybrain.tools.shortcuts import buildNetwork
from pybrain.supervised.trainers import BackpropTrainer
from pybrain.datasets import SupervisedDataSet
from pybrain.structure import SigmoidLayer
from pybrain.structure import LinearLayer
from pybrain.structure import SoftmaxLayer

from sklearn import linear_model
from sklearn import tree
from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, cross_val_score

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer

import pickle
import json

import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
from sklearn.tree import export_graphviz

pd.set_option('display.precision', 3);
%matplotlib inline

import numpy as np
from scipy.stats import norm
from sklearn.linear_model import LinearRegression
import json

## Загрузка проверочной части таблицы

In [8]:
dataTEST = pd.read_csv("ds_test.csv", # путь к файлу, используй автодотолнение
                   sep=',', # разделитель данных в файле
                   header=0 # номер строки с заголовками. header='None', если заголовки отсутствуют
                  )
dataTEST_Norm = pd.read_csv("dsNorm_test.csv", # путь к файлу, используй автодотолнение
                   sep=',', # разделитель данных в файле
                   header=0 # номер строки с заголовками. header='None', если заголовки отсутствуют
                  )

In [9]:
dataTEST.shape

(5208, 18)

In [10]:
dataTEST.columns

Index([u'Unnamed: 0', u'instant', u'dteday', u'season', u'yr', u'mnth', u'hr',
       u'holiday', u'weekday', u'workingday', u'weathersit', u'temp', u'atemp',
       u'hum', u'windspeed', u'casual', u'registered', u'cnt'],
      dtype='object')

In [11]:
features = [ u'temp', u'hr', u'hum']
target = [u'cnt']

In [12]:
dataTEST[:2]

,Unnamed: 0,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,268,269,2011-01-12,1,0,1,15,0,3,1,1,0.20,0.167,0.47,0.418,1,48,49
1,6494,6495,2011-10-03,4,0,10,4,0,1,1,1,0.34,0.348,0.87,0.104,2,6,8


In [13]:
dataTEST_Norm[:2]

,Unnamed: 0,instant,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,dteday_v,cnt
0,0.574,0.574,0.667,0.0,0.545,0.217,0.0,1.0,0.0,0.0,0.673,0.683,0.750,0.204,0.011,0.014,0.573,0.017
1,0.975,0.975,0.000,0.0,1.000,0.783,0.0,1.0,0.0,0.0,0.245,0.217,0.333,0.574,0.019,0.102,0.975,0.107


In [14]:
dataTEST.describe()

,Unnamed: 0,instant,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
count,5208.000,5208.000,5208.000,5208.000,5208.000,5208.000,5208.000,5208.000,5208.000,5208.000,5208.000,5208.000,5208.000,5208.000,5208.000,5208.000,5208.000
mean,8692.476,8693.476,2.501,0.503,6.534,11.486,0.031,3.011,0.676,1.417,0.497,0.475,0.630,0.191,35.295,150.922,186.217
std,5016.759,5016.759,1.110,0.500,3.453,6.981,0.173,2.018,0.468,0.634,0.194,0.173,0.193,0.124,48.677,150.770,180.914
min,11.000,12.000,1.000,0.000,1.000,0.000,0.000,0.000,0.000,1.000,0.020,0.000,0.100,0.000,0.000,0.000,1.000
25%,4406.250,4407.250,2.000,0.000,4.000,5.000,0.000,1.000,0.000,1.000,0.340,0.333,0.480,0.104,4.000,32.000,38.000
50%,8711.500,8712.500,3.000,1.000,7.000,12.000,0.000,3.000,1.000,1.000,0.500,0.485,0.640,0.194,16.000,111.000,137.000
75%,13001.500,13002.500,3.000,1.000,10.000,18.000,0.000,5.000,1.000,2.000,0.660,0.621,0.790,0.254,49.000,214.000,274.000
max,17378.000,17379.000,4.000,1.000,12.000,23.000,1.000,6.000,1.000,3.000,1.000,1.000,1.000,0.851,362.000,833.000,941.000


In [15]:
x_test = dataTEST.drop(columns=target)
y_test = dataTEST[target]
x_test_Norm = dataTEST_Norm.drop(columns=target)
y_test_Norm = dataTEST_Norm[target]

In [16]:
x_test[:2]

,Unnamed: 0,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered
0,268,269,2011-01-12,1,0,1,15,0,3,1,1,0.20,0.167,0.47,0.418,1,48
1,6494,6495,2011-10-03,4,0,10,4,0,1,1,1,0.34,0.348,0.87,0.104,2,6


In [17]:
x_test_Norm[:2]

,Unnamed: 0,instant,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,dteday_v
0,0.574,0.574,0.667,0.0,0.545,0.217,0.0,1.0,0.0,0.0,0.673,0.683,0.750,0.204,0.011,0.014,0.573
1,0.975,0.975,0.000,0.0,1.000,0.783,0.0,1.0,0.0,0.0,0.245,0.217,0.333,0.574,0.019,0.102,0.975


# Загрузка моделей

## Шкалы стандартизации. Коэфф.

In [169]:
newScaler_hr = MinMaxScaler(feature_range=(0, 1))

newScaler_hr.data_min_    =  [0.]
newScaler_hr.data_max_    =  [23.]
newScaler_hr.min_ =  [0.]
newScaler_hr.scale_ =  [0.04347826]
newScaler_hr.data_range_  =  [23.]

# .transform( [[2]] ) - привести значение 2 к заданному диапазону
print newScaler_hr.transform( [[2]] )

# .inverse_transform( [[0.1428571]] ) - привести значение 0.1428571 к исходному диапазону
print newScaler_hr.inverse_transform( [[0.08695652]])

[[0.08695652]]
[[2.]]


In [170]:
newScaler_temp = MinMaxScaler(feature_range=(0, 1))

newScaler_temp.data_min_    =  [0.02]
newScaler_temp.data_max_    =  [1.]
newScaler_temp.min_ =   [-0.02040816]
newScaler_temp.scale_ =  [1.02040816]
newScaler_temp.data_range_  =  [0.98]


# .transform( [[2]] ) - привести значение 2 к заданному диапазону
print newScaler_temp.transform( [[0.64]] )

# .inverse_transform( [[0.1428571]] ) - привести значение 0.1428571 к исходному диапазону
print newScaler_temp.inverse_transform( [[0.63265306]])

[[0.63265306]]
[[0.64]]


In [171]:

newScaler_cnt = MinMaxScaler(feature_range=(0, 1))

newScaler_cnt.data_min_    =   [1.]
newScaler_cnt.data_max_    =  [759.]
newScaler_cnt.min_ =  [-0.00131926]
newScaler_cnt.scale_ =  [0.00131926]
newScaler_cnt.data_range_  =  [758.]


# .transform( [[2]] ) - привести значение 2000000 к заданному диапазону
print newScaler_cnt.transform( [[168]] )

# .inverse_transform( [[0.1428571]] ) - привести значение 1.0 к исходному диапазону
print newScaler_cnt.inverse_transform( [[0.22031642]])

[[0.22031642]]
[[168.]]


### Входные параметры

In [147]:
x_input_test = x_test[features][40:41]
print x_input_test 

    temp  hr   hum
40  0.52  20  0.77


In [148]:
print x_input_test.values[0][0]
print x_input_test.values[0][1]

print newScaler_hr.transform( [[x_input_test.values[0][0]]] )
print newScaler_hr.inverse_transform(newScaler_hr.transform( [[x_input_test.values[0][0]]] ))

print newScaler_temp.transform( [[x_input_test.values[0][1]]] )
print newScaler_temp.inverse_transform(  newScaler_temp.transform( [[x_input_test.values[0][1]]] ))

0.52
20.0
[[0.0226087]]
[[0.52]]
[[20.38775504]]
[[20.]]


In [149]:
x_input_test2 = x_test[features][555:556]
print x_input_test2 

     temp  hr   hum
555  0.62   4  0.83


### Проверочный выходной параметр

In [150]:
y_test[target][40:41].values[0][0]

198

In [151]:
print newScaler_cnt.transform( [[y_test[target][40:41].values[0][0]]] )

[[0.25989422]]


In [152]:
print newScaler_cnt.inverse_transform(newScaler_cnt.transform( [[y_test[target][40:41].values[0][0]]] ))

[[198.]]


In [153]:
y_test[target][555:556].values[0][0]

6

### Нормализованные входные параметры

In [154]:
x1_input = newScaler_hr.transform( [[x_input_test.values[0][0]]] )
x2_input = newScaler_temp.transform( [[x_input_test.values[0][1]]] )

In [155]:
arr_input = [x1_input[0][0], x2_input[0][0]]
print arr_input

[0.0226086952, 20.38775504]


In [156]:
x1_input2 = newScaler_hr.transform( [[x_input_test2.values[0][0]]] )
x2_input2 = newScaler_temp.transform( [[x_input_test2.values[0][1]]] )

In [157]:
arr_input2 = [x1_input2[0][0], x2_input2[0][0]]
print arr_input2

[0.0269565212, 4.06122448]


## Нейронная сеть. Файл pickle.

### Модель 3 вх., 1 вых.

In [267]:
x_sample_3 = x_test_Norm[features][40:41]
x_sample_3.values[0]

array([0.18367347, 0.30434783, 0.63095238])

In [263]:
x_sample_3_1 = x_test_Norm[features][555:556]
x_sample_3_1.values[0]

array([0.51020408, 0.73913043, 0.51190476])

In [268]:
import pickle
fileName_net3 = "model_Net_3to1_BikeCnt.data_pickle"

with open(fileName_net3, 'rb') as f:
    net3 = pickle.load(f)
print "Результат проверки. Нейронная сеть 3 вх., 1 вых. Файл pickle."
print net3.activate( x_sample_3.values[0] )

Результат проверки. Нейронная сеть 3 вх., 1 вых. Файл pickle.
[0.13538304]


In [317]:
r2_net3 = 0.33

In [269]:
x_sample_3.values[0]

array([0.18367347, 0.30434783, 0.63095238])

In [305]:
x_sample_17 = x_test_Norm[0:1]
x_sample_17.values[0]

array([0.57380053, 0.57380053, 0.66666667, 0.        , 0.54545455,
       0.2173913 , 0.        , 1.        , 0.        , 0.        ,
       0.67346939, 0.68338834, 0.75      , 0.20372208, 0.01089918,
       0.01356852, 0.57260274])

### Результат работы модели на проверочных параметрах

In [306]:
print net3.activate( x_sample_3.values[0] )

[0.13538304]


In [307]:
print newScaler_cnt.inverse_transform([net3.activate( x_sample_3.values[0] )])

[[103.62044039]]


In [308]:
y_check_net3 = newScaler_cnt.inverse_transform([net3.activate( x_sample_3.values[0] )])

In [274]:
print newScaler_cnt.inverse_transform([net3.activate( x_sample_3_1.values[0]  )])

[[319.09215061]]


In [275]:
y_check2_net3 = newScaler_cnt.inverse_transform([net3.activate( x_sample_3_1.values[0]  )])

### Модель 17 вх., 1 вых.

In [313]:
# сохраним модель нейнронной сети
import pickle
# имя файла
fileName_net17 = "model_Net_17to1.data_pickle"

with open(fileName_net17, 'rb') as f:
    net17 = pickle.load(f)
print "Результат проверки. Нейронная сеть 3 вх., 1 вых. Файл pickle."
print net17.activate( x_sample_17.values[0] )

Результат проверки. Нейронная сеть 3 вх., 1 вых. Файл pickle.
[0.09811498]


### Результат работы модели на проверочных параметрах

In [277]:
x_input_testAll = x_test_Norm[40:41]
print x_input_testAll.values [0]

[0.84747509 0.84747509 1.         0.         0.90909091 0.30434783
 0.         0.33333333 1.         0.         0.18367347 0.21661166
 0.63095238 0.20372208 0.02452316 0.51289009 0.84931507]


In [278]:
x_input_testAll2 = x_test_Norm[555:556]
print x_input_testAll2.values [0]

[0.92534066 0.92534066 1.         0.         1.         0.73913043
 0.         0.33333333 1.         0.         0.51020408 0.53333333
 0.51190476 0.27779156 0.11716621 0.94979647 0.9260274 ]


In [279]:
y_test_Norm[40:41]

,cnt
40,0.509


In [280]:
print net17.activate( x_input_testAll.values[0] )

[0.42684479]


In [316]:
r2_net17 = 0.91

In [281]:
print net17.activate( x_input_testAll2.values[0] )

[0.94372073]


In [282]:
print newScaler_cnt.inverse_transform([net17.activate( x_input_testAll.values[0] )])

[[324.54864752]]


In [283]:
y_check_net17 = newScaler_cnt.inverse_transform([net17.activate( x_input_testAll.values[0] )])

In [284]:
print newScaler_cnt.inverse_transform([net17.activate( x_input_testAll2.values[0] )])

[[716.34097138]]


In [285]:
y_check2_net17 = newScaler_cnt.inverse_transform([net17.activate( x_input_testAll2.values[0] )])

## Линейная регрессия. Коэфф.

In [172]:
model_LinRegrMult           = linear_model.LinearRegression()
model_LinRegrMult.coef_      = np.array([ 0.250885, -0.09054081, 0.04830667])
model_LinRegrMult.intercept_ = np.array([0.08743233])
y_pred_LinRegrMult_CLEAR           = model_LinRegrMult.predict(  x_test_Norm[features]  )


r2_Lin = metrics.r2_score(y_test_Norm, y_pred_LinRegrMult_CLEAR)
mse_Lin = metrics.mean_squared_error(y_test_Norm, y_pred_LinRegrMult_CLEAR)

print "R2   = ",r2_Lin
print "MAE  = ",mse_Lin, newScaler_cnt.inverse_transform( [[ r2_Lin ]])
print "RMSE  = {} {:,.3f} ".format( mse_Lin,newScaler_cnt.inverse_transform( [[ r2_Lin ]])[0][0])

R2   =  -0.17196600056242506
MAE  =  0.07098996513011369 [[-129.35034835]]
RMSE  = 0.0709899651301 -129.350 


In [173]:
print "Результат проверки. Линейная регрессия. Коэфф."
print "Проверочное ", y_test_Norm.iloc[0:1].values
print "Вычисленное ", y_pred_LinRegrMult_CLEAR[1] 

print "Проверочное ", newScaler_cnt.inverse_transform( [[ 0.114 ]])
print "Вычисленное ", newScaler_cnt.inverse_transform( [[ 0.09379015 ]])



Результат проверки. Линейная регрессия. Коэфф.
Проверочное  [[0.0171504]]
Вычисленное  0.09411775260573797
Проверочное  [[87.4120795]]
Вычисленное  [[72.09299911]]


## Линейная регрессия. Файл pickle.

In [174]:
import pickle
fileName_LinRegrMult = "model_LinRegrMult.data_pickle"

with open(fileName_LinRegrMult, 'rb') as f:
    model_LinRegrMult = pickle.load(f)

y_pred_LinRegrMult_CLEAR = model_LinRegrMult.predict(  x_test_Norm[features]  )
print "Результат проверки. Линейная регрессия. Файл pickle."

print "y_pred Исходной, ", "y_pred Загруженной, ", "delta, ", "delta < 0.000001"

for i in range (0, 10):
    delta = (y_test_Norm.iloc[i:i+1].values  - y_pred_LinRegrMult_CLEAR[i] ) 
    print y_test_Norm.iloc[i:i+1].values , y_pred_LinRegrMult_CLEAR[i], delta, delta < 0.000001

Результат проверки. Линейная регрессия. Файл pickle.
y_pred Исходной,  y_pred Загруженной,  delta,  delta < 0.000001
[[0.0171504]] [0.2402063] [[-0.2230559]] [[ True]]
[[0.10686016]] [0.31622845] [[-0.20936829]] [[ True]]
[[0.01978892]] [0.04213348] [[-0.02234456]] [[ True]]
[[0.2585752]] [0.30922432] [[-0.05064912]] [[ True]]
[[0.82453826]] [0.46972174] [[0.35481652]] [[False]]
[[0.69261214]] [0.15867231] [[0.53393982]] [[False]]
[[0.21635884]] [0.45247233] [[-0.23611349]] [[ True]]
[[0.0171504]] [0.27687071] [[-0.25972031]] [[ True]]
[[0.33773087]] [0.44682706] [[-0.10909619]] [[ True]]
[[0.39841689]] [0.25852098] [[0.13989591]] [[False]]


In [175]:
print model_LinRegrMult.predict( [ x_sample_3.values[0]])

[[0.23336]]


In [176]:
print model_LinRegrMult.predict( [ x_sample_3_1.values[0]] )

[[0.16299687]]


In [190]:
print newScaler_cnt.inverse_transform(model_LinRegrMult.predict( [x_sample_3_1.values[0]] ))

[[124.55173843]]


In [207]:
y_check_linRegr = newScaler_cnt.inverse_transform(model_LinRegrMult.predict( [x_sample_3.values[0]] ))

In [192]:
print newScaler_cnt.inverse_transform(model_LinRegrMult.predict( [ x_sample_3.values[0]]))

[[177.88704228]]


In [239]:
y_check2_linRegr = newScaler_cnt.inverse_transform(model_LinRegrMult.predict( [ x_sample_3_1.values[0]]))

## Дерево регрессии. Файл pickle.

In [225]:
import pickle
fileName_treeRegr_d7 = "model_treeRegr_d7.data_pickle"

with open(fileName_LinRegrMult, 'rb') as f:
    treeRegr_d7 = pickle.load(f)

y_pred_treeRegr_d7 = treeRegr_d7.predict(  x_test_Norm[features]  )
print "Результат проверки. Дерево регрессии"

print "y_pred Исходной, ", "y_pred Загруженной, ", "delta, ", "delta < 0.000001"

for i in range (0, 10):
    delta = (y_test_Norm.iloc[i:i+1].values  - y_pred_treeRegr_d7[i] ) 
    print y_test_Norm.iloc[i:i+1].values , y_pred_treeRegr_d7[i], delta, delta < 0.000001

Результат проверки. Дерево регрессии
y_pred Исходной,  y_pred Загруженной,  delta,  delta < 0.000001
[[0.0171504]] [0.2402063] [[-0.2230559]] [[ True]]
[[0.10686016]] [0.31622845] [[-0.20936829]] [[ True]]
[[0.01978892]] [0.04213348] [[-0.02234456]] [[ True]]
[[0.2585752]] [0.30922432] [[-0.05064912]] [[ True]]
[[0.82453826]] [0.46972174] [[0.35481652]] [[False]]
[[0.69261214]] [0.15867231] [[0.53393982]] [[False]]
[[0.21635884]] [0.45247233] [[-0.23611349]] [[ True]]
[[0.0171504]] [0.27687071] [[-0.25972031]] [[ True]]
[[0.33773087]] [0.44682706] [[-0.10909619]] [[ True]]
[[0.39841689]] [0.25852098] [[0.13989591]] [[False]]


In [296]:
print "Дерево решений. Регрессия. Усеченный диапазон."
mae_tree = metrics.mean_absolute_error(y_test_Norm, y_pred_treeRegr_d7)
r2_tree  = metrics.r2_score           (y_test_Norm, y_pred_treeRegr_d7)

print "MAE - средняя абсолютная ошибка: {:>7,.0f} дерево регр. ".format( mae_tree)
print "R2  - коэфф. детерминации:       {:>7,.3f} дерево регр. ".format( r2_tree )

Дерево решений. Регрессия. Усеченный диапазон.
MAE - средняя абсолютная ошибка:       0 дерево регр. 
R2  - коэфф. детерминации:         0.335 дерево регр. 


In [226]:
treeRegr_d7.predict([x_sample_3_1.values[0]])

array([[0.16299687]])

In [227]:
treeRegr_d7.predict([x_sample_3.values[0]])

array([[0.23336]])

In [235]:
newScaler_cnt.inverse_transform(treeRegr_d7.predict([x_sample_3.values[0]]))

array([[177.88704228]])

In [234]:
y_check_tree = newScaler_cnt.inverse_transform(treeRegr_d7.predict([x_sample_3.values[0]]))

In [236]:
newScaler_cnt.inverse_transform(treeRegr_d7.predict([x_sample_3_1.values[0]]))

array([[124.55173843]])

In [237]:
y_check2_tree = newScaler_cnt.inverse_transform(treeRegr_d7.predict([x_sample_3_1.values[0]]))

# Сравнение результатов

In [318]:
y1 = y_test[target][40:41].values[0][0]
y2 = y_test[target][555:556].values[0][0]
print"y Исходное                                  ", y1, '\n' 
print "Сеть 3вх. 1 вых.:      ", "y_pred Загруженной  ", y_check_net3[0][0],  "   Разница ", abs(y_check_net3[0][0]-y1), " Предполагаемая ошибка {:>10,.2f}".format(y1*0.1218),"R2   = {:>7,.3f} ".format(r2_net3),'\n'
print "Сеть 17вх. 1 вых.:     ", "y_pred Загруженной  ", y_check_net17[0][0],  "  Разница ", abs(y_check_net17[0][0]-y1), " Предполагаемая ошибка {:>10,.2f}".format(y1*0.1065),"R2   = {:>7,.3f} ".format(r2_net17),'\n'
print "Дерево регрессии:      ", "y_pred Загруженной  ", y_check_tree[0][0],  "  Разница ", abs(y_check_tree[0][0]-y1), " Предполагаемая ошибка {:>10,.2f}".format(y1*0.1214),"R2   = {:>7,.3f} ".format(r2_tree),'\n'
print "Множ.лин.регрессия:    ", "y_pred Загруженной  ", y_check_linRegr[0][0],  "  Разница ", abs(y_check_linRegr[0][0]-y1), " Предполагаемая ошибка {:>10,.2f}".format(y1*0.1216), " R2   = {:>7,.3f}".format(r2_Lin), '\n'
print  '\n' 
print"y Исходное             ", y_test[target][555:556].values[0][0], '\n' 
print "Сеть 3вх. 1 вых.:      ", "y_pred Загруженной  ", y_check2_net3[0][0],  "  Разница ", abs(y_check2_net3[0][0]-y2), "  Предполагаемая ошибка {:>10,.2f}".format(y2*0.1218),'\n'
print "Сеть 17вх. 1 вых.:     ", "y_pred Загруженной  ", y_check2_net17[0][0],  "   Разница ", abs(y_check2_net17[0][0]-y2),"   Предполагаемая ошибка {:>10,.2f}".format(y2*0.1065), '\n'
print "Дерево регрессии:      ", "y_pred Загруженной  ", y_check2_tree[0][0],  "  Разница ", abs(y_check2_tree[0][0]-y2),"  Предполагаемая ошибка {:>10,.2f}".format(y1*0.1214), '\n'
print "Множ.лин.регрессия:    ", "y_pred Загруженной  ", y_check2_linRegr[0][0],  "  Разница ", abs(y_check2_linRegr[0][0]-y2), "   Предполагаемая ошибка {:>10,.2f}".format(y1*0.1216),'\n'

y Исходное                                   198 

Сеть 3вх. 1 вых.:       y_pred Загруженной   103.62044038738487    Разница  94.37955961261513  Предполагаемая ошибка      24.12 R2   =   0.330  

Сеть 17вх. 1 вых.:      y_pred Загруженной   324.5486475157762   Разница  126.54864751577622  Предполагаемая ошибка      21.09 R2   =   0.910  

Дерево регрессии:       y_pred Загруженной   177.88704228005577   Разница  20.112957719944234  Предполагаемая ошибка      24.04 R2   =   0.335  

Множ.лин.регрессия:     y_pred Загруженной   177.88704228005577   Разница  20.112957719944234  Предполагаемая ошибка      24.08  R2   =  -0.172 



y Исходное              6 

Сеть 3вх. 1 вых.:       y_pred Загруженной   319.09215061014913   Разница  313.09215061014913   Предполагаемая ошибка       0.73 

Сеть 17вх. 1 вых.:      y_pred Загруженной   716.3409713770011    Разница  710.3409713770011    Предполагаемая ошибка       0.64 

Дерево регрессии:       y_pred Загруженной   124.55173842957551   Разница 

# Сопровождающая информация по моделям

In [35]:
dictModel_linear   = dict()
dictModel_tree     = dict()
dictModel_neuroNet = dict()

In [292]:
dictModel_linear["name"]      = u"Множественная. v2"
dictModel_linear["typeML"]      = u"Линейная регрессия"
dictModel_linear["r2"]        = 0.33
dictModel_linear["mae"]       = 0.15
dictModel_linear["mse"]       = 0.017
dictModel_linear["features"]  = ["hr","hum","temp"]
dictModel_linear["target"]    = ["cnt"]
dictModel_linear["dataNormalised"]    = True


dictModel_tree["name"]      = u"Дерево регрессии"
dictModel_tree["typeML"]      = u"Дерево регрессии"
dictModel_tree["r2"]        = 0.61
dictModel_tree["mae"]       = 0
dictModel_tree["mse"]       = 0
dictModel_tree["features"]  = ["hr","hum","temp"]
dictModel_tree["target"]    = ["cnt"]
dictModel_tree["dataNormalised"]    = True


dictModel_neuroNet["name"]      = u"Нейросеть 3вх, 1 вых. 26 итераций"
dictModel_neuroNet["typeML"]      = u"Нейронная сеть"
dictModel_neuroNet["r2"]        = 0.45
dictModel_neuroNet["mae"]       = 0.05
dictModel_neuroNet["mse"]       = 0.005
dictModel_neuroNet["features"]  = ["hr","hum","temp"]
dictModel_neuroNet["target"]    = ["cnt"]
dictModel_neuroNet["dataNormalised"]    = True

# Выводы

Для выполнения лабораторной работы данные были нормализованы с помощью функции OrdinalEncoder().
В ходе выполнения лабораторной работы были построены и обучены следующие виды моделей:

- Нейронная сеть (классический перцептрон). Количество узлов входного слоя равно количеству независимых переменных (3). Количество узлов выходного слоя равно количеству целевых переменных (1). Нейросеть была обучена за 26 итерации. Был получен коэффициент детерминации, равный 0.91, mae - 0.05.

- Модель линейной множественной регрессии была построена на следующих независимых переменных: время суток (hr), влажность (hum), температура (temp) и целевой переменной количество клиентов (cnt). Был получен коэффициент детерминации, равный 0.33, mae - 0.15. 

- Дерево регрессии глубиной 7 имеет коэффициент детерминации 0.61, mae - 0.